In [15]:
import os
import numpy as np
import cv2
import keras
from keras.models import Sequential
from keras.layers import *
from keras import backend as k
from keras import optimizers
from keras.models import Model
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.engine.topology import Layer
from PIL import Image
from glob import glob
from time import time

In [2]:
BATCH_SIZE = 32
batch_test = 4
root_dir = os.path.normpath(os.path.join(os.path.dirname(os.path.realpath('__file__'))))
data_directory = os.path.join(root_dir, 'lfw-deepfunneled') # Make a directory to store the data and enter it here.
                    # We will be using a smaller dataset (LFW) than the one used in the paper (CelebA) for computational resource considerations.
                    # Download it from http://vis-www.cs.umass.edu/lfw/lfw-deepfunneled.tgz.
resize_dir = os.path.join(root_dir, 'Resized_images')
#device = -1
#lambda_ = {'feature': 1., 'pixel': 1., 'total_variation': 1e-5}
model_directory = os.path.join(root_dir, 'NIPS_model_ass1') # Make a directory to store the models and enter it here. Move Vgg4Layers.npz to the model directory.
outsize = (96, 96)

In [ ]:
# Resizing and saving each portrait in the data directory

'''iterator = 0

for root, dirs, files in os.walk(data_directory):
    for fname in files:

        filepath = os.path.join(root, fname)
        
        img = cv2.imread(filepath)
        
        img1 = cv2.resize(img, (96, 96))
        
        cv2.imwrite(os.path.join(resize_dir, '%r_1.jpg' %iterator), img1)
        
        iterator += 1'''

In [3]:
def render(img_rgb):
    """
    Applies pencil sketch effect to an RGB image
    :param img_rgb: RGB image to be processed
    :returns: Processed RGB image
    """
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (21, 21), 0, 0)
    img_blend = cv2.divide(img_gray, img_blur, scale=256)
    
    return img_blend

In [4]:
def img_load(img):
    
    t_ = []
    x_ = []
    for i in img:
        t = np.asarray(Image.open(i).convert('RGB').resize((96, 96), Image.LANCZOS), 'f').transpose(2, 0, 1)
        x = render(np.asarray(Image.open(i).convert('RGB').resize((96, 96), Image.LANCZOS), 'f'))[None]
        
        t_.append(t)
        x_.append(x)
    
    return np.asarray(t_), np.asarray(x_)

In [5]:
data_file = sorted(glob('{}/*.jpg'.format(resize_dir)))
training_set = img_load(data_file[:int(.64 * len(data_file))])
validation_set = img_load(data_file[int(.64 * len(data_file)) : int(.8 * len(data_file))])
test_set = img_load(data_file[int(.8 * len(data_file) ):])

batches = int(len(training_set)/BATCH_SIZE)

In [6]:
def batch_generator():
    
    while True:

        for batch in range(batches):
            
            data = training_set[1][batch * BATCH_SIZE: (batch + 1) * BATCH_SIZE]
            target = training_set[0][batch * BATCH_SIZE: (batch + 1) * BATCH_SIZE]
                
            yield (data, target)

In [7]:
def residual_block(y):
    shortcut = y
    
    y = Conv2D(128, 3, strides = 1, padding = 'same', data_format='channels_first')(y)
    y = BatchNormalization(axis = 1, scale = None)(y)
    y = Activation('relu')(y)
    
    y = Conv2D(128, 3, strides = 1, padding = 'same', data_format='channels_first')(y)
    y = BatchNormalization(axis = 1, scale = None)(y)
    y = Activation('relu')(y)

    y = Add()([shortcut, y])

    return y

In [11]:
input1 = Input(shape = (1, 96, 96))

model = (ZeroPadding2D(4, data_format='channels_first'))(input1)

model = (Conv2D(32, 9, strides = 1, data_format='channels_first'))(model)

model = (BatchNormalization(axis = 1, scale = None))(model)

model = (Activation('relu'))(model)

model = (Conv2D(64, 3, strides = 2, padding = 'same', data_format='channels_first'))(model)

model = (BatchNormalization(axis = 1, scale = None))(model)

model = (Activation('relu'))(model)

model = (Conv2D(128, 3, strides = 2, padding = 'same', data_format='channels_first'))(model)

model = (BatchNormalization(axis = 1, scale = None))(model)

model = (Activation('relu'))(model)

res1 = residual_block(model)

res2 = residual_block(res1)

res3 = residual_block(res2)

res4 = residual_block(res3)

res5 = residual_block(res4)

deconv1 = Conv2DTranspose(64, 3, strides = 2, padding = 'same', data_format='channels_first')(res5)

bn1 = BatchNormalization(axis = 1, scale = None)(deconv1)

relu1 = Activation('relu')(bn1)

deconv2 = Conv2DTranspose(32, 3, strides = 2, padding = 'same', data_format='channels_first')(relu1)

bn2 = BatchNormalization(axis = 1, scale = None)(deconv2)

relu2 = Activation('relu')(bn2)

zero_pad = ZeroPadding2D(4, data_format='channels_first')(relu2)

conv_fin = Conv2D(3, 9, strides = 1, data_format='channels_first')(zero_pad)

bn3 = BatchNormalization(axis = 1, scale = None)(conv_fin)

out = Activation('tanh')(bn3)

model_ = Model(inputs=[input1], outputs=out)

In [12]:
adam_ = optimizers.Adam(lr = 1e-3, beta_1=0.9, beta_2=0.999, epsilon=1e-8)

In [21]:
def root_mean_squared_error(y_true, y_pred):
    return k.sqrt(k.mean(k.square(y_pred - y_true), axis=-1))

In [23]:
model_.compile(loss = root_mean_squared_error, optimizer = adam_)

In [13]:
model_.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1, 96, 96)    0                                            
__________________________________________________________________________________________________
zero_padding2d_3 (ZeroPadding2D (None, 1, 104, 104)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_15 (Conv2D)              (None, 32, 96, 96)   2624        zero_padding2d_3[0][0]           
__________________________________________________________________________________________________
batch_normalization_17 (BatchNo (None, 32, 96, 96)   96          conv2d_15[0][0]                  
__________________________________________________________________________________________________
activation

In [16]:
tensorboard = TensorBoard(log_dir="Sketch_invert_logs/{}".format(time()), histogram_freq = 1, write_graph=True, write_images = True)

In [18]:
checkpointer = ModelCheckpoint(filepath='checkpoint_sketch.h5', save_best_only=True)

In [19]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-7)

In [20]:
callback_list = [tensorboard, checkpointer, reduce_lr]

In [26]:
x_val = []
t_val = []

for i in range(len(validation_set[1])):
    x_val.append(validation_set[1][i])
    t_val.append(validation_set[0][i])
    
valx = np.asarray(x_val)
valt = np.asarray(t_val)

In [ ]:
model.fit_generator(batch_generator(), steps_per_epoch = batches, epochs=10, shuffle = True, 
                    validation_data = (valx, valt), callbacks = callback_list)